<center><img src="images/DLI Header.png" alt="Header" width="400"></center>

# 6.0 Different Neural Networks
### INTERNET CONNECTION TO JETSON REQUIRED

In the examples presented so far, the object detector is a ResNet10 network trained to detect four different objects (vehicle, bicycle, person, road sign). In many cases, you may wish to use a different network for other tasks. DeepStream natively supports other networks including YOLO, SSD, and Faster-RCNN. For experienced developers, additional networks can be integrated with DeepStream through TensorRT.  

To use these 3rd-party networks, Internet access is required to download network weights and configuration files.  An easy way to accomplish this is through the Ethernet port on the Jetson Nano.  If you've installed a WiFi connection on your Jetson, that will also work just fine. 

In this notebook, you'll learn how to incorporate a YOLO network into a DeepStream app.

<img src="images/05_example.png" alt="YOLO network pipeline">

**[6.1 YOLO](#6.1-YOLO)**<br>
&nbsp; &nbsp; &nbsp;[6.1.1 Set Up YOLO with DeepStream SDK](#6.1.1-Set-Up-YOLO-with-DeepStream-SDK)<br>
&nbsp; &nbsp; &nbsp;[6.1.2 Exercise: Run a YOLO Application](#6.1.2-Exercise:-Run-a-YOLO-Application)<br>
&nbsp; &nbsp; &nbsp;[6.1.3 Exercise: Annotate Your Own Video with YOLO](#6.1.3-Exercise:-Annotate-Your-Own-Video-with-YOLO)<br>
**[6.2 Faster-RCNN](#6.2-Faster-RCNN)**<br>
**[6.3 SSD](#6.3-SSD)**<br>

# 6.1 YOLO
The YOLO, or "You Only Look Once", network employs a single convolutional network that simultaneously predicts multiple bounding boxes and class probabilities for those boxes.  DeepStream SDK includes support for YOLO (using C++) in the `/opt/nvidia/deepstream/deepstream/sources/objectDetector_Yolo` directory. Navigate there now using the JupyterLab file browser (the `deepstream` shortcut link provided takes you directly to `/opt/nvidia/deepstream/deepstream`). The `labels.txt` file contains the ordered list of 80 objects recognized by this network!

In [ ]:
# Set some path locations for readability
YOLO_DETECTOR = '/opt/nvidia/deepstream/deepstream/sources/objectDetector_Yolo'
STREAMS = '/opt/nvidia/deepstream/deepstream/samples/streams'
DLI_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/dli_apps'

In [ ]:
# Explore the contents objectDetector_Yolo
# There are a number of configuration files to choose from
!ls -l $YOLO_DETECTOR

In [ ]:
# Take a look at the 80 objects that YOLO can detect
!cat $YOLO_DETECTOR/labels.txt

## 6.1.1 Set Up YOLO with DeepStream SDK
The instructions for setting up YOLO for DeepStream are provided in the [README](deepstream/sources/objectDetector_Yolo/README) file in the `objectDetector_Yolo` folder. A variation of the `deepstream-test3` sample, named `deepstream-test3-mp4-out-yolo` has been provided for the course.  It is the same as the `deepstream-test3-mp4-out` application used in a previous notebook with one important difference: it uses the YOLO config file instead of the ResNet10 sample config file.  This parameter is set in [deepstream_test3_mp4_out.py](deepstream/sources/deepstream_python_apps/dli_apps/deepstream-test3-mp4-out-yolo/deepstream_test3_mp4_out.py) as a property of the `nvinfer` plugin, `pgie`:

```python
pgie.set_property('config-file-path', "config_infer_primary_yoloV3_tiny.txt")
```

The config file is a copy from the YOLO folder, with some important modifications.  The paths used in config, model, label, and library paths have been updated.  If these are incorrect, the application will fail.

In [ ]:
# View the differences between the original YOLO config file and the one we will use
!diff $YOLO_DETECTOR/config_infer_primary_yoloV3_tiny.txt \
    $DLI_APPS/deepstream-test3-mp4-out-yolo/config_infer_primary_yoloV3_tiny.txt

We must still download the YOLO weights and build the model.  Follow the steps below to set up YOLO on your Jetson Nano.  Your Jetson Nano must be connected to the Internet.
1. Download the weights and configuration for the model

In [ ]:
# Download YOLO weights from the Internet
!cd $YOLO_DETECTOR && ./prebuild.sh

2. Check the CUDA version on your machine

In [ ]:
!cat /usr/local/cuda/version.txt

3. If the CUDA Version does not match the variable CUDA_VER in the following cell, modify the value to match your CUDA version correctly.  Only do this to one decimal point, for example "CUDA_VER=10.2"
4. Execute the following cell to build the YOLO engine

In [ ]:
# TODO Verify the CUDA_VER value
# Build YOLO engine
CUDA_VER = 10.2
!cd $YOLO_DETECTOR/nvdsinfer_custom_impl_Yolo/ \
    && CUDA_VER=$CUDA_VER make

## 6.1.2 Exercise: Run a YOLO Application
Run the following sample application to see YOLO in action.  After running the following cells, DeepStream will save the result to `out.mp4`.  Download `out.mp4` by right clicking it in JupyterLab file browser and select "Download". Then play it on your machine.

In [ ]:
# Check usage of the test3 mp4 app with the help option
!cd $DLI_APPS/deepstream-test3-mp4-out-yolo \
    && python3 deepstream_test3_mp4_out.py --help

In [ ]:
# Run the app with YOLO
OUTPUT_PATH_EX1 = '/dli/task/nb6_out.mp4'
!cd $DLI_APPS/deepstream-test3-mp4-out-yolo \
    && python3 deepstream_test3_mp4_out.py -o $OUTPUT_PATH_EX1 -i \
        file://$STREAMS/sample_720p.h264

In [ ]:
# Watch the saved video
import os
from IPython.display import Video

video_path = os.path.relpath(OUTPUT_PATH_EX1)
Video(video_path, width = 600, height = 300)

#### How did you do?
The YOLO network has more objects it can detect, including a "bus", which was not detected specifically in the ResNet10 sample detector.  If you see something like this image when you play your saved file, you've successfully used the YOLO network! 

<img src="images/05_yolo.png" alt="file output YOLO">

## 6.1.3 Exercise: Annotate Your Own Video with YOLO
Since the sample file, `sample_720p.h264`, has mainly cars and people in the scene, you won't see all YOLO has to offer with the 80 objects it can detect.  Try annotating your own video instead of using the provided sample:

1. Import your own video file: drag-and-drop it from your machine to JupyterLab file browser.
2. Change the `file://your/file/path.mp4` placeholder path in the cell below to the input path of your imported file. 
3. Execute the cell to run the app
4. Watch the saved video to see the result

In [ ]:
# TODO Insert the name of your uploaded video to replace "my_video_in.mp4"
INPUT_PATH = '/dli/task/my_video_in.mp4'
# Run the app with YOLO
OUTPUT_PATH_EX2 = '/dli/task/nb6_my_video_out.mp4'
!cd $DLI_APPS/deepstream-test3-mp4-out-yolo \
    && python3 deepstream_test3_mp4_out.py -o $OUTPUT_PATH_EX2 -i \
        file://$INPUT_PATH

In [ ]:
# Watch the saved video
import os
from IPython.display import Video

video_path = os.path.relpath(OUTPUT_PATH_EX2)
Video(video_path, width = 600, height = 300)

# 6.2 Faster-RCNN

This sample uses TensorRT plugins, performs inference, and implements a fused custom layer for end-to-end inferencing of a Faster R-CNN model. Specifically, this sample demonstrates the implementation of a Faster R-CNN network in TensorRT, performs a quick performance test in TensorRT, implements a fused custom layer, and constructs the basis for further optimization.

Navigate to `deepstream/sources/objectDetector_FasterRCNN` using the JupyterLab file browser. Click the README for detailed instructions on DeepStream prerequisites and usage.

# 6.3 SSD
    
The SSD, or "Single Shot Detector", network performs the task of object detection and localization in a single forward pass of the network. This network is built using the VGG network as a backbone. Unlike Faster R-CNN, SSD completely eliminates proposal generation and subsequent pixel or feature-resampling stages.  It encapsulates all computation in a single network. This makes SSD straightforward to integrate into systems that require a detection component. 

Navigate to `deepstream/sources/objectDetector_SSD` using the JupyterLab file browser. Click the README for detailed instructions on DeepStream prerequisites and usage.

<h2 style="color:green;">Congratulations!</h2>

You ran a DeepStream app using a different network. <br>
Move on to [7.0 Live Stream](./07_DSWebcam.ipynb).

<center><img src="images/DLI Header.png" alt="Header" width="400"></center>